In [5]:
import torch 
from torch import nn
import numpy as np

from typing import List

In [3]:
from src.env import PuzzleEnv

env_config = {
    "puzzle_name": "cube_2x2x2",
    "num_envs": 10,
    "max_steps": 20,
    "reset_config": {
        "sampler": "uniform",
        "shuffle_range": [1, 5],
    },
    "reward_config": {
        "success": "dynamic",
        "time": -1,
    },
    "state_repr": "color",
}

env = PuzzleEnv(env_config)

Loaded cube_2x2x2: 12 actions, 24 states


For DQN we'll use a simple FFN similar to dist_predictor

In [7]:
class DQN(nn.Module):
    def __init__(self, inp: int, oup: int, units: List[int]):
        # TODO: we can optionally add normalization layers
        super().__init__()
        layers = []
        prev = inp
        for num_units in units:
            layers.append(nn.Linear(prev, num_units))
            layers.append(nn.ReLU())
            prev = num_units

        layers.append(nn.Linear(units[-1], oup))

        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)


net = DQN(inp=env.state_size, oup=env.num_actions, units=[128, 128])

In [8]:
net

DQN(
  (layers): Sequential(
    (0): Linear(in_features=24, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=12, bias=True)
  )
)